In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from stop_words import get_stop_words
import nltk
import numpy as np
import random
import string
import re
"""
- En utilisant la fonction `open` (et en précisant le paramètre `encoding` adéquat!), importez le texte dans une chaîne de caractères que vous nommerez `texte`.  
- Procédez à quelques nettoyages : 
    - passez le texte en minuscules.  
    - vérifiez s'il n'y pas d'acronyme pouvant fausser notre tokenization en phrases, si oui, remplacez-le par autre chose.  
    - le virus est appelé covid-19 ou coronavirus, faites en sorte qu'un terme unique soit utilisé.   
- Avec la fonction `nltk.sent_tokenize`, passez votre chaîne de caractères `texte` en une liste de phrases que vous appelerez `phrases_token`.  
- Il y a beaucoup de questions dans cette liste, or on veut des réponses. Supprimez-les.
- Récupérez un vecteur de stop words français avec la fonction `get_stop_words` du module `stop_words` 
"""
f=open('./infos_corona.txt','r',errors = 'ignore', encoding = "utf8")
texte=f.read()
texte = texte.lower()
texte = re.sub(r"\ufeff", " ", texte)
texte = re.sub(r"\[.{1,2}\]", " ", texte)
texte = re.sub(r"\n", " ", texte)

texte = re.sub(' covid-19| virus| covid\W| sars-cov|ncov', ' coronavirus', texte)
texte = re.sub('coronavirus coronavirus', 'coronavirus', texte)
texte = re.sub('mort(\w){0,3}|deces|deced(\w){1,5}', 'deces', texte)
texte = re.sub('medec(\w){1,5}|medic{1,5}', 'medical', texte) 

texte = re.sub('c\'est', 'est', texte)
texte = re.sub('/s',' ',texte)
#texte = re.sub(f"[{string.punctuation}]", " ", texte)
texte = re.sub('[éèê]', 'e', texte)
texte = re.sub('[àâ]', 'a', texte)
texte = re.sub('[ô]', 'o', texte)

phrases_token = nltk.sent_tokenize(texte, language = "french")

for i in sorted(range(len(phrases_token)), reverse = True):
    if re.search(r"\?", phrases_token[i]):
        del phrases_token[i]
        
#phrases_token = list(set(phrases_token))
french_stop_words = get_stop_words('french')
#texte

lemmer = nltk.stem.WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

# un exemple : 
#LemNormalize(texte)


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
#liste = ['avon', 'do', 'doi', 'e', 'fuss', 'no', 'pa', 's', 'sou']
french_stop_words = french_stop_words #+ liste
tfidf = TfidfVectorizer(stop_words=french_stop_words)
phrases_tf = tfidf.fit_transform(phrases_token)


def response(phrase_user, tfidf=tfidf, phrases_tf=phrases_tf):
    # on a besoin de passer la chaîne de caractère dans une liste :
    phrase_user = [phrase_user]
    # On calcule les valuers TF-IDF pour la phrase de l'utilisateur
    user_tf = tfidf.transform(phrase_user)
    # on calcule la similarité entre la question posée par l'utilisateur
    # et l'ensemble des phrases de la page wiki
    similarity = cosine_similarity(user_tf, phrases_tf).flatten()
    # on sort l'index de la phrase étant la plus similaire
    index_max_sim = np.argmax(similarity)
    # Si la similarité max ets égale à 0 == pas de correspondance trouvée
    if (similarity[index_max_sim] == 0):
        robo_response = "Je suis désolé car je ne connais pas la réponse"
    # Sinon, on sort la phrase correspondant le plus : 
    else:
        robo_response = phrases_token[index_max_sim]
    return robo_response

In [6]:
GREETING_INPUTS = ("hello", "bonjour", "salut")
GREETING_RESPONSES = ["hello","bonjour", "je suis ravi de discuter avec vous"]
def greeting(sentence): 
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [7]:
flag=True
print("COROBOT: Mon nom est Corobot. Posez moi votre question ou si vous voulez arretez dites simplement bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if re.search(r'bye|à bientôt|au revoir', user_response):
        flag=False
        print("COROBOT: Au revoir, prenez soin de vous et rester chez vous bordel !")
    else:
        if(user_response=='merci' or user_response=='merci' ):
            flag=False
            print("COROBOT: Avec plaisir.")
        else:
            if greeting(user_response) != None :
                print("COROBOT: " + greeting(user_response))
            else:
                print("COROBOT: " + response(user_response))

COROBOT: Mon nom est Corobot. Posez moi votre question ou si vous voulez arretez dites simplement bye!


 gestes barrieres


COROBOT: ce sont les gestes barrieres et la distanciation sociale qui sont efficaces.


 puis je sortir voir des amis ?


COROBOT: vous pouvez sortir avec vos enfants ou seul mais pas retrouver des amis.


...

COROBOT: uniquement pour vous occuper de personnes vulnerables comme par exemple les personnes handicapees et en vous rappelant que les seniors sont les personnes les plus vulnerables : il faut les proteger le plus possible de tout contact et porter une attention encore renforcee aux gestes barrieres.


 bonjour


COROBOT: hello


 merci


COROBOT: Avec plaisir.
